### Package imports

In [1]:
# Import packages
import re
import os
import copy
from pathlib import Path
import numpy as np
from numpy.random import MT19937
from numpy.random import RandomState, SeedSequence
import pandas as pd
import random
import datetime
import time
from pdb import set_trace
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.mixture import BayesianGaussianMixture
import gc

### Set seed function

In [2]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    rs = RandomState(MT19937(SeedSequence(seed)))

### File path

In [3]:
# File path
raw_data_path = '/home/ec2-user/MLNotebook/Datasets/Test set raw data/'
output_path = '/home/ec2-user/MLNotebook/Datasets/'

### Testing set generation

#### Johansson

In [4]:
# Johansson data set
# Open the proteomics data and only keep genes (rows) that are fully quantified
J_PFP = 'jo_protein_log2.csv' # proteomics file path, normalized by pool, log2 transformed.
J_PD = pd.read_csv(raw_data_path+J_PFP)

# Data set wrangling
J_PD.index = J_PD.loc[:,'Unnamed: 0']
J_PD = J_PD.loc[:,J_PD.columns!='Unnamed: 0']
J_PD.dropna(inplace=True)

# Put values of each column in the DataFrame into a list
values = np.sort(J_PD.values.flatten().tolist())

#Find the 2.5 and 97.5 percentile
percentile_high = np.percentile(values, 97.5)
percentile_low = np.percentile(values, 2.5)

# Use the percentile for normalization
J_PD = (J_PD - percentile_low) / (percentile_high - percentile_low)

# Open the mRNA data and only keep genes (rows) that are fully quantified
J_MFP = 'jo_mrna_dropna.csv' # mRNA file path, gene centric median normalized, log2 transformed
J_MD = pd.read_csv(raw_data_path + J_MFP)

# Data set wrangling
J_MD.index = J_MD.loc[:,'Unnamed: 0']
J_MD = J_MD.loc[:,J_MD.columns!='Unnamed: 0']
J_MD = J_MD.drop_duplicates()
J_MD.dropna(inplace=True)

# Put values of each column in the DataFrame into a list
values = np.sort(J_MD.values.flatten().tolist())

#Find the 2.5 and 97.5 percentile
percentile_high = np.percentile(values, 97.5)
percentile_low = np.percentile(values, 2.5)

# Use the percentile for normalization
J_MD = (J_MD - percentile_low) / (percentile_high - percentile_low)

In [5]:
# Open the localization data
LFP = 'SubCellBarcode.MCF7.txt'
LD = pd.read_csv(filepath_or_buffer=raw_data_path+LFP,sep='\t')

# Data set wrangling
LD.index = LD.loc[:,'Protein']
LD = LD.loc[:,LD.columns!='Protein']

# Remove unclassified class
NotUnclassInd = LD.loc[:,'Localization'] != 'Unclassified'
LD = LD.loc[NotUnclassInd,:]

In [6]:
# Keep only genes (rows) are presented in proteome, mRNA and localization data sets
IntersectingGenes = [value for value in J_PD.index if ((value in J_MD.index) & (value in LD.index))]
J_PD = J_PD.loc[IntersectingGenes,:]
J_MD = J_MD.loc[IntersectingGenes,:]
LD = LD.loc[IntersectingGenes,:]

# Sanity check for the number of genes in each dataframe
print('Johansson')
print(len(J_PD.index))
print(len(J_MD.index))
print(len(LD.index))

# Encode the localization labels
encoder = LabelEncoder()

# Fit the encoder and transform the labels to integers from LD, the label df
J_labels = encoder.fit_transform(LD.values.ravel())

# Save gene names for later use
J_gene_names = J_PD.index.tolist()

Johansson
7989
7989
7989


In [7]:
# Export the labels for use
LD.to_csv(Path(output_path+'Johansson_Localization.csv'))

#### Mertins

In [8]:
# Mertins data set
# Open the proteomics data and only keep genes (rows) that are fully quantified
M_PFP = 'me_protein_dropna.csv' # proteomics file path, normalized by pool, log2 transformed.
M_PD = pd.read_csv(raw_data_path+M_PFP)

# Data set wrangling
M_PD.index = M_PD.loc[:,'Unnamed: 0']
M_PD = M_PD.loc[:,M_PD.columns!='Unnamed: 0']
M_PD.dropna(inplace=True)

# Put values of each column in the DataFrame into a list
values = np.sort(M_PD.values.flatten().tolist())

#Find the 2.5 and 97.5 percentile
percentile_high = np.percentile(values, 97.5)
percentile_low = np.percentile(values, 2.5)

# Use the percentile for normalization
M_PD = (M_PD - percentile_low) / (percentile_high - percentile_low)

# Open the mRNA data and only keep genes (rows) that are fully quantified
M_MFP = 'me_rna_dropna.csv' # mRNA file path, gene centric median normalized, log2 transformed
M_MD = pd.read_csv(raw_data_path + M_MFP)

# Data set wrangling
M_MD.index = M_MD.loc[:,'Unnamed: 0']
M_MD = M_MD.loc[:,M_MD.columns!='Unnamed: 0']
M_MD = M_MD.drop_duplicates()
M_MD.dropna(inplace=True)

# Put values of each column in the DataFrame into a list
values = np.sort(M_MD.values.flatten().tolist())

#Find the 2.5 and 97.5 percentile
percentile_high = np.percentile(values, 97.5)
percentile_low = np.percentile(values, 2.5)

# Use the percentile for normalization
M_MD = (M_MD - percentile_low) / (percentile_high - percentile_low)

In [9]:
# Open the localization data
LFP = 'SubCellBarcode.MCF7.txt'
LD = pd.read_csv(filepath_or_buffer=raw_data_path+LFP,sep='\t')

# Data set wrangling
LD.index = LD.loc[:,'Protein']
LD = LD.loc[:,LD.columns!='Protein']

# Remove unclassified class
NotUnclassInd = LD.loc[:,'Localization'] != 'Unclassified'
LD = LD.loc[NotUnclassInd,:]

In [10]:
# Keep only genes (rows) are presented in proteome, mRNA and localization data sets
IntersectingGenes = [value for value in M_PD.index if ((value in M_MD.index) & (value in LD.index))]
M_PD = M_PD.loc[IntersectingGenes,:]
M_MD = M_MD.loc[IntersectingGenes,:]
LD = LD.loc[IntersectingGenes,:]

# Sanity check for the number of genes in each dataframe
print('Mertin')
print(len(M_PD.index))
print(len(M_MD.index))
print(len(LD.index))

# Fit the encoder and transform the labels to integers from LD, the label df
M_labels = encoder.fit_transform(LD.values.ravel())

# Save gene names for later use
M_gene_names = M_PD.index.tolist()

Mertin
5076
5076
5076


In [11]:
# Export the labels for use
LD.to_csv(Path(output_path+'Mertins_Localization.csv'))

### Bayesian inference to generate synthetic data

In [12]:
Set = 'Protein' # Change it to 'Protein' if only generating proteome synthetic data
Bayesian = True
Canvas_Size = 18

#### Johansson

In [13]:
# Johansson Bayesian inference
# Ensure reproducibility
set_seed(43)

if Bayesian:

    if Set == 'Protein' or Set == 'Protein + mRNA':
        # Fit the dataset to Bayesian Gaussian Mixture Model
        PD_bgm = BayesianGaussianMixture(n_components=15, 
            weight_concentration_prior_type='dirichlet_process', 
            weight_concentration_prior=0.5, 
        random_state=42) # Assuming the maximum number of clusters in dataset is 25
        J_PD_T = J_PD.T
        PD_bgm.fit(J_PD_T)
    
        # Generate X new synthetic tumors, result is an array
        if Set == 'Protein':
            J_synthetic_PD, _ = PD_bgm.sample(int(Canvas_Size*Canvas_Size-J_PD.columns.size))
        elif Set == 'Protein + mRNA':
            J_synthetic_PD, _ = PD_bgm.sample(int((Canvas_Size*Canvas_Size-J_PD.columns.size*2)/2))
    
        # Transpose back before merging
        J_synthetic_PD = J_synthetic_PD.T

        # Convert the result to a DataFrame
        J_synthetic_PD = pd.DataFrame(J_synthetic_PD.tolist(), index=J_PD.index)

        # Merge the synthetic data with the original data
        J_PD = pd.concat([J_PD, J_synthetic_PD], axis=1)

        # Sanity check for the number of tumors in each dataframe
        
        if Set == 'Protein + mRNA':
            assert len(J_PD.columns) == Canvas_Size*Canvas_Size // 2
        else:
            assert len(J_PD.columns) == Canvas_Size*Canvas_Size

        print(f'{Set} - Bayesian Johansson testing PD: {len(J_PD.columns)}')
      
    if Set == 'mRNA' or Set == 'Protein + mRNA':
        # Fit the dataset to Bayesian Gaussian Mixture Model
        MD_bgm = BayesianGaussianMixture(n_components=25, 
            weight_concentration_prior_type='dirichlet_process', 
            weight_concentration_prior=0.5, 
        random_state=43) # Assuming the maximum number of clusters in dataset is 25
        J_MD_T = J_MD.T
        MD_bgm.fit(J_MD_T)

        # Generate X new synthetic tumors, result is an array
        if Set == 'mRNA':
            J_synthetic_MD, _ = MD_bgm.sample(int(Canvas_Size*Canvas_Size-J_MD.columns.size))
        if Set == 'Protein + mRNA':
            J_synthetic_MD, _ = MD_bgm.sample(int((Canvas_Size*Canvas_Size-J_MD.columns.size*2)/2))

        # Transpose back before merging
        J_synthetic_MD = J_synthetic_MD.T

        # Convert the result to a DataFrame
        J_synthetic_MD = pd.DataFrame(J_synthetic_MD.tolist(), index=J_MD.index)

        # Merge the synthetic data with the original data
        J_MD = pd.concat([J_MD, J_synthetic_MD], axis=1)
         
        # Sanity check for the number of tumors in each dataframe
        if Set == 'Protein + mRNA':
            assert len(J_MD.columns) == Canvas_Size*Canvas_Size // 2
        else:
            assert len(J_MD.columns) == Canvas_Size*Canvas_Size
        print(f'{Set} - Bayesian Johansson testing MD: {len(J_MD.columns)}')

else:
    print('No synthetic data generated')

MemoryError: Unable to allocate 7.13 GiB for an array with shape (15, 7989, 7989) and data type float64

In [ ]:
# export the synthetic data
if Set == 'Protein + mRNA':
    J_PD.to_csv(Path(output_path + 'J_prot+mRNA_PD_synthetic.csv'))
    J_MD.to_csv(Path(output_path + 'J_prot+mRNA_MD_synthetic.csv'))
elif Set == 'Protein':
    J_PD.to_csv(Path(output_path + 'J_prot_PD_synthetic.csv'))
elif Set == 'mRNA':
    J_MD.to_csv(Path(output_path + 'J_prot_MD_synthetic.csv'))

In [ ]:
gc.collect()

#### Mertins

In [ ]:
# Mertins Bayesian inference
set_seed(Seed)

if Bayesian:

    if Set == 'Protein' or Set == 'Protein + mRNA':
        # Fit the dataset to Bayesian Gaussian Mixture Model
        PD_bgm = BayesianGaussianMixture(n_components=25, 
            weight_concentration_prior_type='dirichlet_process', 
            weight_concentration_prior=0.5, 
        random_state=42) # Assuming the maximum number of clusters in dataset is 25
        M_PD_T = M_PD.T
        PD_bgm.fit(M_PD_T)
    
        # Generate X new synthetic tumors, result is an array
        if Set == 'Protein':
            M_synthetic_PD, _ = PD_bgm.sample(int(Canvas_Size*Canvas_Size-M_PD.columns.size))
        elif Set == 'Protein + mRNA':
            M_synthetic_PD, _ = PD_bgm.sample(int((Canvas_Size*Canvas_Size-M_PD.columns.size*2)/2))
    
        # Transpose back before merging
        M_synthetic_PD = M_synthetic_PD.T

        # Convert the result to a DataFrame
        M_synthetic_PD = pd.DataFrame(M_synthetic_PD.tolist(), index=M_PD.index)

        # Merge the synthetic data with the original data
        M_PD = pd.concat([M_PD, M_synthetic_PD], axis=1)

        # Sanity check for the number of tumors in each dataframe
        
        if Set == 'Protein + mRNA':
            assert len(M_PD.columns) == Canvas_Size*Canvas_Size // 2
        else:
            assert len(M_PD.columns) == Canvas_Size*Canvas_Size

        print(f'{Set} - Bayesian Mertin testing PD: {len(M_PD.columns)}')
      
    if Set == 'mRNA' or Set == 'Protein + mRNA':
        # Fit the dataset to Bayesian Gaussian Mixture Model
        MD_bgm = BayesianGaussianMixture(n_components=25, 
            weight_concentration_prior_type='dirichlet_process', 
            weight_concentration_prior=0.5, 
        random_state=43) # Assuming the maximum number of clusters in dataset is 25
        M_MD_T = M_MD.T
        MD_bgm.fit(M_MD_T)

        # Generate X new synthetic tumors, result is an array
        if Set == 'mRNA':
            M_synthetic_MD, _ = MD_bgm.sample(int(Canvas_Size*Canvas_Size-M_MD.columns.size))
        if Set == 'Protein + mRNA':
            M_synthetic_MD, _ = MD_bgm.sample(int((Canvas_Size*Canvas_Size-MD.columns.size*2)/2))

        # Transpose back before merging
        M_synthetic_MD = M_synthetic_MD.T

        # Convert the result to a DataFrame
        M_synthetic_MD = pd.DataFrame(M_synthetic_MD.tolist(), index=M_MD.index)

        # Merge the synthetic data with the original data
        M_MD = pd.concat([M_MD, M_synthetic_MD], axis=1)
         
        # Sanity check for the number of tumors in each dataframe
        if Set == 'Protein + mRNA':
            assert len(M_MD.columns) == Canvas_Size*Canvas_Size // 2
        else:
            assert len(M_MD.columns) == Canvas_Size*Canvas_Size
        print(f'{Set} - Bayesian Mertin testing MD: {len(M_MD.columns)}')

else:
    print('No synthetic data generated')

In [33]:
# Export
if Set == 'Protein + mRNA':
    M_PD.to_csv(Path(output_path + 'M_prot+mRNA_PD_synthetic.csv'))
    M_MD.to_csv(Path(output_path + 'M_prot+mRNA_MD_synthetic.csv'))
elif Set == 'Protein':
    M_PD.to_csv(Path(output_path + 'M_prot_PD_synthetic.csv'))
elif Set == 'mRNA':
    M_MD.to_csv(Path(output_path + 'M_prot_MD_synthetic.csv'))